In [8]:
def get_json_depth(json_obj, current_path='', depth=0, result=None):
    if result is None:
        result = {}
    
    if isinstance(json_obj, dict):
        for key, value in json_obj.items():
            new_path = f"{current_path}.{key}" if current_path else key
            get_json_depth(value, new_path, depth + 1, result)
    elif isinstance(json_obj, list):
        for index, item in enumerate(json_obj):
            new_path = f"{current_path}[{index}]"
            get_json_depth(item, new_path, depth, result)
    else:  # 基本类型
        result[current_path] = (json_obj, depth)
    
    return result

{'name': ('John', 1),
 'age': (30, 1),
 'address.street': ('21 2nd Street', 2),
 'address.city': ('New York', 2),
 'address.coordinates.latitude': (40.7128, 3),
 'address.coordinates.longitude': (-74.006, 3),
 'hobbies[0]': ('reading', 1),
 'hobbies[1]': ('cycling', 1),
 'hobbies[2].sport': ('swimming', 2)}

In [9]:
import geopandas as gpd
import pandas as pd
from shapely import Polygon
xy = [(108.7245,30.9241),(108.7237,30.9241),(108.723,30.925),(108.7245,30.9241)]
gdf = gpd.GeoDataFrame(pd.DataFrame({'bh':'1'},index=[0]),geometry=[Polygon(xy)],crs="EPSG:4490")
gdf.to_file(r"E:\工作文档\测试导出数据\test.shp")

In [11]:
aa = [3,6,7]
aa[:-1]

[3, 6]